In [101]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import scipy.signal as sig
import pywt

### Set up the Device

In [102]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


### Load the Data

In [115]:
dataset_path = './data/'
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### K-Fold

In [116]:
# some major changes here for the Train_Val_Data function
def Train_Val_Data(X_train_valid, y_train_val):
    '''
    split the train_valid into k folds (we fix k = 5 here)
    return: list of index of train data and val data of k folds
    train_fold[i], val_fold[i] is the index for training and validation in the i-th fold 

    '''
    fold_idx = []
    train_fold = []
    val_fold = []
    train_val_num = X_train_valid.shape[0]
    fold_num = int(train_val_num / 5)
    perm = np.random.permutation(train_val_num)
    for k in range(5):
        fold_idx.append(np.arange(k*fold_num, (k+1)*fold_num, 1))
    for k in range(5):
        val_fold.append(fold_idx[k])
        count = 0
        for i in range(5):
            if i != k:
                if count == 0:
                    train_idx = fold_idx[i]
                else:
                    train_idx = np.concatenate((train_idx, fold_idx[i]))
                count += 1
        train_fold.append(train_idx)

    return train_fold, val_fold

In [117]:
train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

### Data Augmentation Functions

### 1. Window Data

In [30]:
def window_data(X, y, p, window_size, stride):
  '''
  X (a 3-d tensor) of size (#trials, #electrodes, #time series)
  y (#trials,): label 
  p (#trials, 1): person id

  X_new1: The first output stacks the windowed data in a new dimension, resulting 
    in a 4-d tensor of size (#trials x #electrodes x #windows x #window_size).
  X_new2: The second option makes the windows into new trails, resulting in a new
    X tensor of size (#trials*#windows x #electrodes x #window_size). To account 
    for the larger number of trials, we also need to augment the y data.
  y_new: The augmented y vector of size (#trials*#windows) to match X_new2.
  p_new: The augmented p vector of size (#trials*#windows) to match X_new2
 
  '''
  num_sub_trials = int((X.shape[2]-window_size)/stride)
  X_new1 = np.empty([X.shape[0],X.shape[1],num_sub_trials,window_size])
  X_new2 = np.empty([X.shape[0]*num_sub_trials,X.shape[1],window_size])
  y_new = np.empty([X.shape[0]*num_sub_trials])
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      for k in range(num_sub_trials):
        X_new1[i,j,k:k+window_size]    = X[i,j,k*stride:k*stride+window_size]
        X_new2[i*num_sub_trials+k,j,:] = X[i,j,k*stride:k*stride+window_size]
        y_new[i*num_sub_trials+k] = y[i]
        p_new[i*num_sub_trials+k] = p[i]
  return X_new1, X_new2, y_new, p_new

### 2. STFT

In [51]:
# Function that computes the short-time fourier transform of the data and returns the spectrogram
def stft_data(X, window, stride):
    '''
    Inputs:
    X - input data, last dimension is one which transform will be taken across.
    window - size of sliding window to take transform across
    stride - stride of sliding window across time-series

    Returns:
    X_STFT - Output data, same shape as input with last dimension replaced with two new dimensions, F x T.
            where F = window//2 + 1 is the frequency axis
            and T = (input_length - window)//stride + 1, similar to the formula for aconvolutional filter.
    t - the corresponding times for the time axis, T
    f - the corresponding frequencies on the frequency axis, F.

    reshape X_STFT (N, C, F, T) to (N, C*F, T) to fit the input of rnn

    Note that a smaller window means only higher frequencies may be found, but give finer time resolution.
    Conversely, a large window gives better frequency resolution, but poor time resolution.

    '''
    noverlap = window-stride
    #print(noverlap)
    if noverlap < 0 :
        print('Stride results in skipped data!')
        return
    f, t, X_STFT = sig.spectrogram(X,nperseg=window,noverlap=noverlap,fs=250, return_onesided=True)
    N, C, F, T = X_STFT.shape
    X_STFT = X_STFT.reshape(N, C*F, T)
    return X_STFT

### 3. CWT

In [61]:
def cwt_data(X, num_levels, top_scale=3):
    '''
    Takes in data, computes CWT using the mexican hat or ricker wavelet using scipy
    Also takes in the top scale parameter.  I use logspace, so scale goes from 1 -> 2^top_scale with num_levels steps.
    Appends to the data a new dimension, of size 'num_levels'
    New dimension corresponds to wavelet content at num_levels different scalings (linear)
    also returns the central frequencies that the scalings correspond to
    input data is N x C X T
    output data is N x C x T x F
    note: CWT is fairly slow to compute

    # EXAMPLE USAGE
    test, freqs = cwt_data(X_train_valid[0:5,:,:],num_levels=75,top_scale=4)
    '''
    scales = np.logspace(start=0,stop=top_scale,num=num_levels)
    out = np.empty((X.shape[0],X.shape[1],X.shape[2],num_levels))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            coef = sig.cwt(X[i,j,:],sig.ricker,scales)
            out[i,j,:] = coef.T
    freqs = pywt.scale2frequency('mexh',scales)*250
    N, C, T, F = out.shape
    X_CWT = np.transpose(out, (0,1,3,2)).reshape(N, C*F, T)
    return X_CWT

In [71]:
def Aug_Data(X, y, p, aug_type=None, window_size=None, window_stride=None, stft_size=None, stft_stride=None, cwt_level=None, cwt_scale=None):
    if aug_type == None:
        X_train, y_train, p_train = X, y, p
    elif aug_type == "window":
        _, X_train, y_train, p_train = window_data(X, y, p, window_size, window_stride)
    elif aug_type == "stft":
        X_train = stft_data(X, stft_size, stft_stride)
        y_train, p_train = y, p
    elif aug_type == 'cwt':
        X_train = cwt_data(X, cwt_level, cwt_scale)
        y_train, p_train = y, p
    
    return X_train, y_train, p_train

### Customized Dataset

In [118]:
X_train_valid[train_fold[0]].shape

(1692, 22, 1000)

In [125]:
class EEG_Dataset(Dataset):
    '''
    use use fold_idx to instantiate different train val datasets for k-fold cross validation

    '''
    def __init__ (self, train_fold, val_fold, fold_idx=0, mode='train'):
        if mode == 'train':
            self.X = X_train_valid[train_fold[fold_idx]]
            self.y = y_train_valid[train_fold[fold_idx]] - 769
            self.p = person_train_valid[train_fold[fold_idx]]
            
        elif mode == 'val':
            self.X = X_train_valid[val_fold[fold_idx]]
            self.y = y_train_valid[val_fold[fold_idx]] - 769
            self.p = person_train_valid[val_fold[fold_idx]]         

        elif mode == 'test':
            self.X = X_test
            self.y = y_test - 769        
            self.p = person_test

    def __len__(self):
        return (self.X.shape[0])
    
    def __getitem__(self, idx):
        '''
        X: (augmented) time sequence 
        y: class label
        p: person id

        '''
        X = torch.from_numpy(self.X[idx,:,:]).float()
        y = torch.tensor(self.y[idx]).long()
        p = torch.from_numpy(self.p[idx,:]).long()     
        sample = {'X': X, 'y': y, 'p':p}

        return sample

### Define Basic LSTM

In [130]:
class LSTMnet(nn.Module):
    '''
    Create Basic LSTM:
    2 layers

    TODO: make number of layers, dropout, activation function, regularization all params
    see ex: https://blog.floydhub.com/gru-with-pytorch/
    '''

    def __init__(self, input_size, hidden_size, output_dim):
        super(LSTMnet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=2, dropout=0.5)
        self.fc = nn.Linear(hidden_size, output_dim)
    
    def forward(self, x, h=None):
        if type(h) == type(None):
            out, hn = self.rnn(x)
        else:
            out, hn = self.rnn(x, h.detach())
        out = self.fc(out[-1, :, :])
        return out

### Define Basic GRU